# Problem Statement :- We are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

Summarize the review.

The summary should be at most 3 lines.


Highlight both the positives and negatives


Display the overall sentiment of the review (positive, negative, neutral)


Display a list of 3 - 5 emotions expressed by the customer in the review


If the sentiment is positive or neutral write an email and thank them for the review


If the sentiment is negative apologize and write an email with an appropriate response


Try to get the response in a nice structured format using an output parser

In [3]:
!pip install langchain
!pip install langchain_community
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.1 MB/s eta 0:00:00


In [4]:
from getpass import getpass
OPENAI_API_KEY=getpass("Enter api key :-")

Enter api key :-··········


In [5]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [6]:
from langchain_openai import ChatOpenAI
chatgpt=ChatOpenAI(model="gpt-4o-mini",temperature=0)

In [7]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [8]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
class CustomerReviewOutput(BaseModel):
  summary: str=Field(description="A brief summary of the customer review with maximum 3 lines.")
  positives:list=Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
  negatives:list=Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
  sentiment: str=Field(description="One word showing the sentiment of the review - positive, negative or neutral")
  emotions: list=Field(description="A list of 3-5 emotions expressed by the customer in the review")
  email: str=Field(description="Detailed email to the customer based on the sentiment")

parser=PydanticOutputParser(pydantic_object=CustomerReviewOutput)


In [9]:
prompt_text = """
You are an intelligent assistant. Your task is to analyze the following customer review and provide a structured response.

Review:
{review}

Instructions:
1. Summarize the review in no more than 3 lines.
2. Highlight both positive and negative aspects.
3. Determine the overall sentiment: Positive, Negative, or Neutral.
4. Identify 3 to 5 emotions expressed by the customer.
5. Based on the sentiment:
   - If Positive or Neutral: write a polite thank-you email.
   - If Negative: write a polite apology email with an appropriate response.
6. The email should be signed by a customer service agent.

{format_instructions}
"""


In [10]:
from langchain_core.prompts import PromptTemplate
prompt=PromptTemplate(template=prompt_text,
                      input_variables=["review"],
                      partial_variables={"format_instructions":parser.get_format_instructions})

In [11]:
chain=prompt | chatgpt | parser

In [12]:
input=[{"review":review}for review in reviews]   #very importatn to tag our customer review with review key for mapping purpose

In [13]:
responses=chain.map().invoke(input)

In [14]:
for res,review in zip(responses,reviews):
  print(" Customer review :- "+review)
  print("Output :-")
  for k,v in res.dict().items():
    print(f'{k}:-\n{v}')

 Customer review :- 
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    
Output :-
summary:-
The customer is very pleased with the Bluetooth speaker, praising its sound quality, waterproof feature, and compact design. They also appreciated the early shipping and secure packaging.
positives:-
['Impressive sound quality with clear audio and good bass', 'Waterproof feature tested successfully', 'Compact size with powerful volume', 'Great price for high-quality sound', 'Early shipping in secure packaging']
negatives:-
[]
sentiment:-
Positive
emotions:-
['Satisfaction', 'Excitement', 'Relief', 'Happ

<ipython-input-14-1657896850>:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for k,v in res.dict().items():


In [15]:
import pandas as pd

In [17]:
data=pd.DataFrame(respones.dict() for respones in responses)

<ipython-input-17-1498303870>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  data=pd.DataFrame(respones.dict() for respones in responses)


In [19]:
data

,summary,positives,negatives,sentiment,emotions,email
0,The customer is very pleased with the Bluetoot...,[Impressive sound quality with clear audio and...,[],Positive,"[Satisfaction, Excitement, Relief, Happiness, ...","Dear Customer,\n\nThank you so much for your w..."
1,The customer is pleased with their new gaming ...,"[Satisfying click of the keys, Vibrant LED col...",[],Positive,"[Satisfaction, Excitement, Contentment]","Dear Customer,\n\nThank you for your wonderful..."
2,The customer is disappointed with the wireless...,[Arrived on time],"[Sound constantly cuts out, Uncomfortable fit ...",Negative,"[Disappointment, Frustration, Dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
3,The customer is disappointed with the tablet s...,[Arrived promptly],"[Wobbles with the slightest touch, Angles do n...",Negative,"[Disappointment, Frustration, Dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
4,"The customer is dissatisfied with the blender,...",[],"[Struggles with tough foods, Incredibly noisy,...",Negative,"[Frustration, Disappointment, Anger]","Dear Customer,\n\nThank you for taking the tim..."
